In [21]:

import pandas as pd
import numpy as np
df = pd.read_csv("training_data.csv")

# Map labels to integers if needed
df['increase_stock_label'] = df['increase_stock'].map({
    "low_bike_demand": 0,
    "high_bike_demand": 1
})

df["month"] = df["month"] -1

y = df['increase_stock_label'] 
X = df.drop(columns=["increase_stock", "increase_stock_label"])

X["hour_sin"] = np.sin(2 * np.pi * X["hour_of_day"] / 24)
X["hour_cos"] = np.cos(2 * np.pi * X["hour_of_day"] / 24)

X["day_sin"] = np.sin(2 * np.pi * X["day_of_week"] / 7)
X["day_cos"] = np.cos(2 * np.pi * X["day_of_week"] / 7)

X["month_sin"] = np.sin(2 * np.pi * X["month"] / 12)
X["month_cos"] = np.cos(2 * np.pi * X["month"] / 12)

X = X.drop(columns=["hour_of_day", "day_of_week", "month"])

np.random.seed(3)
N = len(X)
M = np.ceil(0.8 * N).astype(int)  # Number of training data

idx = np.random.permutation(N)

X_train, X_test = X.iloc[idx[:M]], X.iloc[idx[M:]]
y_train, y_test = y.iloc[idx[:M]], y.iloc[idx[M:]]




In [22]:
import xgboost as xgb
from sklearn.metrics import accuracy_score


model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train,
    y_train,
#    eval_set=[(X_test, y_test)],
    verbose=False
)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Test accuracy:", accuracy)


Test accuracy: 0.925


In [23]:
X_train.shape

(1280, 18)

In [24]:
X_train

,holiday,weekday,summertime,temp,dew,humidity,precip,snow,snowdepth,windspeed,cloudcover,visibility,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos
147,0,1,0,2.7,-2.1,70.55,0.300,0,0.30,10.7,100.0,16.0,0.866025,0.500000,0.433884,-0.900969,-0.500000,8.660254e-01
938,0,1,1,20.6,18.9,90.26,0.143,0,0.00,16.0,100.0,16.0,-0.965926,-0.258819,-0.433884,-0.900969,0.500000,-8.660254e-01
492,1,1,0,1.6,-8.3,47.80,0.000,0,0.65,36.4,47.8,16.0,-0.258819,0.965926,0.000000,1.000000,0.000000,1.000000e+00
1274,0,0,1,15.8,12.7,81.68,0.000,0,0.00,15.5,94.3,16.0,0.707107,0.707107,-0.974928,-0.222521,-1.000000,-1.836970e-16
73,0,1,1,9.2,7.8,91.28,0.000,0,0.00,8.9,90.1,7.2,0.965926,0.258819,-0.433884,-0.900969,0.866025,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,0,0,0,13.3,-2.0,34.67,0.000,0,0.00,12.8,84.5,16.0,-0.965926,0.258819,-0.974928,-0.222521,0.500000,8.660254e-01
260,0,1,1,24.8,22.8,88.80,0.000,0,0.00,7.5,36.3,16.0,0.258819,0.965926,0.433884,-0.900969,-0.500000,-8.660254e-01
810,0,1,1,23.2,22.1,93.68,0.000,0,0.00,9.0,79.3,16.0,0.965926,0.258819,0.781831,0.623490,-0.500000,-8.660254e-01
305,0,1,1,28.2,12.2,37.09,0.000,0,0.00,22.8,82.9,16.0,-0.258819,-0.965926,0.433884,-0.900969,0.500000,-8.660254e-01
